In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [8]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [9]:
df = fetch_hourly_rides(12)

2025-03-04 05:27:32,578 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 05:27:32,581 INFO: Initializing external client
2025-03-04 05:27:32,587 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 05:27:33,653 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.98s) 


In [10]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-03 23:00:00+00:00,123,0
1,2025-03-03 21:00:00+00:00,255,1
2,2025-03-03 22:00:00+00:00,247,0
3,2025-03-03 23:00:00+00:00,6,0
4,2025-03-04 06:00:00+00:00,100,139
...,...,...,...
3760,2025-03-04 05:00:00+00:00,35,1
3761,2025-03-04 06:00:00+00:00,145,1
3762,2025-03-03 21:00:00+00:00,125,26
3763,2025-03-04 07:00:00+00:00,123,1


In [11]:
df_pred = fetch_predictions(12)

current time is: 2025-03-04 10:27:40.644205+00:00
2025-03-04 07:00:00+00:00
2025-03-04 05:27:40,644 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 05:27:40,652 INFO: Initializing external client
2025-03-04 05:27:40,652 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 05:27:41,626 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [12]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,127,0.0,2025-03-04 10:00:00+00:00
1,146,2.0,2025-03-04 10:00:00+00:00
2,106,0.0,2025-03-04 10:00:00+00:00
3,260,2.0,2025-03-04 10:00:00+00:00
4,174,0.0,2025-03-04 10:00:00+00:00
...,...,...,...
246,90,52.0,2025-03-04 10:00:00+00:00
247,95,0.0,2025-03-04 10:00:00+00:00
248,48,73.0,2025-03-04 10:00:00+00:00
249,255,3.0,2025-03-04 10:00:00+00:00


In [ ]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [ ]:
merged_df

In [ ]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [ ]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

In [ ]:
merged_df

In [ ]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [ ]:
mae_by_hour["MAE"].mean()